# FAOstat: Crops and livestock products
[_Source data_](https://www.fao.org/faostat/en/#data)

## Parameters

In [ ]:
dest_dir = "/tmp/faostat_fbs"

## Imports

In [ ]:
import zipfile
import tempfile
import shutil
from pathlib import Path
import os

import requests
import pandas as pd

from owid.catalog import Dataset, Table, frames, LocalCatalog
from owid.walden import Catalog
from etl.paths import DATA_DIR
from etl.steps.data.converters import convert_walden_metadata

## Fetch walden dataset

In [ ]:
walden_ds = Catalog().find_one("faostat", "2021-04-09", "faostat_FBS")

In [ ]:
walden_ds

## Make a dataset

In [ ]:
ds = Dataset.create_empty(dest_dir)
ds.metadata = convert_walden_metadata(walden_ds)
ds.metadata.short_name = "faostat_fbs"
ds.save()

## Unzip to temp directory

In [ ]:
tmp_dir = tempfile.mkdtemp()

In [ ]:
z = zipfile.ZipFile(walden_ds.local_path)
z.extractall(tmp_dir)
# print(os.listdir(tmp_dir))
(filename,) = list(filter(lambda x: "(Normalized)" in x, os.listdir(tmp_dir)))
filename

## Load dataset
### Data file

In [ ]:
df = pd.read_csv(os.path.join(tmp_dir, filename), encoding="latin-1")

In [ ]:
df.head()

## Clean dataset

### Sanity checks

In [ ]:
# Check nulls
df.isnull().any()

In [ ]:
# def check_unique_pairs(df, name_1, name_2):
#     if not (
#         (df.groupby(name_1)[name_2].nunique() != 1).any()
#         and (df.groupby(name_2)[name_1].nunique() != 1).any()
#     ):
#         raise ValueError(
#             f"Some `{name_1}` may have multiple `{name_2}` values (or opposite)."
#         )


# fields = ["Item", Element"]
# for field in fields:
#     check_unique_pairs(df, field, f"{field} Code")

In [ ]:
x = df.groupby(["Element", "Unit"])["Element Code"].nunique()
if (x > 1).any():
    raise ValueError(f"Element-Unit not unique!")

### Drop columns and set Index

In [ ]:
df.drop(columns=["Area", "Item", "Element", "Year Code"], inplace=True)

In [ ]:
df.set_index(
    ["Area Code", "Item Code", "Element Code", "Year", "Flag"],
    inplace=True,
    verify_integrity=True,
)

## Add tables

In [ ]:
from owid.catalog import utils

t = Table(df)
t.metadata.short_name = "bulk"
ds.add(utils.underscore_table(t))

## Cleanup

In [ ]:
shutil.rmtree(tmp_dir)